# HW10 Plane Crash

Name: Eric Wu

### Imports

In [133]:
#bigdir = '/Users/jkinser/Documents/CoursesKinser/CDSCardinal/Version5/'
#bigdir = '/home/jkinser/Documents/courses/CDSCardinal/Version5/'
bigdir = '/Users/eric/Documents/CDS465/'

pydir = bigdir + 'pysrc/'
datadir = bigdir + 'data/'
popdir = bigdir + 'population/'

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
import sys
import warnings
warnings.filterwarnings('ignore', category=FutureWarning) 

sys.path.append( pydir )
import Main
import detective as dtcv

E = Main.Realm(datadir)
E.ReadAll('/Users/eric/Documents/University/CDS465HW/HW10/planecrash.db')
len(E.persondf)

101916

# Identify missing persons

In [134]:
#import manifest and DNA excel files
manif = pd.read_excel('/Users/eric/Documents/University/CDS465HW/HW10/manifest.xlsx').drop(['level_0','index'],axis=1)
dna = pd.read_excel('/Users/eric/Documents/University/CDS465HW/HW10/dna.xlsx')

In [155]:
dna.columns

Index(['Unnamed: 0', 'nstr', 'mtdna', 'ystr', 'gender', 'approx'], dtype='object')

In [136]:
### Try getting pids and legal parents from social query using manifest

keep = ['pid','birth','knowndeath','firstname','lastname', 'gender', 'hoh','spouse', 'legaldad','legalmom', 'dometh','locv','loch']
temp_manif = pd.DataFrame(data=[], columns=keep)

for row in manif.itertuples():
    temp_manif = pd.concat([temp_manif, dtcv.SocialQuery(E, row.firstname, row.lastname)], ignore_index=True)

temp_manif = temp_manif.query(f'knowndeath == {temp_manif['knowndeath'].max()}').drop_duplicates() #get pids for those on the manifest AND filter for those who's known death was equivalent to the max

temp_manif

,pid,birth,knowndeath,firstname,lastname,gender,hoh,spouse,legaldad,legalmom,dometh,locv,loch,firstnameS,lastnameS,firstnameD,lastnameD,firstnameM,lastnameM
2,89676,7727,8200,Wanetta,Brey,F,0,NaN,NaN,NaN,0,1130.612725,675.235720,NaN,NaN,Dominick,Wesolowski,Joaquina,Wesolowski
3,93245,7874,8200,Rick,Desalvo,M,0,NaN,NaN,NaN,0,1043.292196,1131.402808,NaN,NaN,Hipolito,Desalvo,Eustolia,Desalvo
4,97696,8057,8200,Arlie,Fort,M,0,NaN,NaN,NaN,2,1077.729999,1144.727351,NaN,NaN,Miguel,Fort,Jacki,Fort
5,98712,8097,8200,Carmella,Fort,F,0,NaN,NaN,NaN,3,1077.729999,1144.727351,NaN,NaN,Miguel,Fort,Jacki,Fort
6,89148,7707,8200,Jacki,Fort,F,0,NaN,NaN,NaN,2,1077.729999,1144.727351,NaN,NaN,Kirk,McLendon,Regan,Turman
7,88794,7694,8200,Miguel,Fort,M,0,NaN,NaN,NaN,3,1077.729999,1144.727351,NaN,NaN,Sang,Fort,Sonya,Pompey
8,99761,8134,8200,Nelson,Fort,M,0,NaN,NaN,NaN,2,1077.729999,1144.727351,NaN,NaN,Miguel,Fort,Jacki,Fort
9,98964,8106,8200,Anneliese,Hickmon,F,0,NaN,NaN,NaN,0,573.100734,1298.895688,NaN,NaN,Hobert,Hickmon,Mari,Hickmon
10,89397,7717,8200,Hobert,Hickmon,M,0,NaN,NaN,NaN,0,573.100734,1298.895688,NaN,NaN,Terrell,Hickmon,Susana,Hickmon
11,99480,8124,8200,Hobert,Hickmon,M,0,NaN,NaN,NaN,0,573.100734,1298.895688,NaN,NaN,Hobert,Hickmon,Mari,Hickmon


Nally Quincy, M, 101489 may be our immigrant since his legal D&M don't show up in social query

In [163]:
### Cartesian join; elim mismatches of approx age and gender.

mnfdnadf = pd.merge(manif, dna, how='cross', suffixes=('_mnf', '_dna'))
mnfdnadf.columns

Index(['firstname', 'lastname', 'gender_mnf', 'approx_mnf', 'Unnamed: 0',
       'nstr', 'mtdna', 'ystr', 'gender_dna', 'approx_dna'],
      dtype='object')

In [ ]:
### Filter for gender_mnf == gender_dna & approx_mnf == approx_dna

mnfdnadf.query('gender_mnf == gender_dna and approx_mnf == approx_dna')

(51, 10)

In [137]:
### Try getting dna from our possible vics identified from the social query
"""temp_dna = pd.DataFrame(data=[],columns=['pid','nstr','ystr','mtdna'])

for pid in temp_manif['pid'].values:
    print(pid)
    queried_dna = dtcv.DNAQuery(E,pid)

    if type(queried_dna) == pd.DataFrame:
        temp_dna = pd.concat([temp_dna,queried_dna], ignore_index=True)"""

"temp_dna = pd.DataFrame(data=[],columns=['pid','nstr','ystr','mtdna'])\n\nfor pid in temp_manif['pid'].values:\n    print(pid)\n    queried_dna = dtcv.DNAQuery(E,pid)\n\n    if type(queried_dna) == pd.DataFrame:\n        temp_dna = pd.concat([temp_dna,queried_dna], ignore_index=True)"

In [138]:
### Try matching DNA based off names
for row in manif.itertuples():
    jailrec = dtcv.JailQuery(E, row.firstname, row.lastname)
    print(pd.merge(jailrec, dna.drop(['mtdna', 'ystr'], axis=1), on='nstr') if len(jailrec) > 0 else f"No record found for {row.firstname} {row.lastname}")

  firstname lastname  start   end  \
0   Wanetta     Brey   8048  8113   

                                                                                                                 nstr  \
0  (12,14);(29,30);(9,10);(9,12);(14,15);(6,9);(9,13);(10,13);(17,19);(14,14);(14,18);(11,11);(12,21);(11,11);(24,26)   

  ystr                 mtdna  Unnamed: 0 gender approx  
0       1;3;w1;1;2;2;4;1;1;1          11      F  Adult  
No record found for Rick Desalvo
No record found for Arlie Fort
No record found for Carmella Fort
No record found for Jacki Fort
No record found for Miguel Fort
No record found for Nelson Fort
No record found for Anneliese Hickmon
No record found for Hobert Hickmon
No record found for Hobert Hickmon
No record found for Luise Hickmon
No record found for Mari Hickmon
No record found for Ashanti McGary
No record found for Quincy Nally
No record found for Shalom Spindler


Wannetta Brey seems to have a viable match dna match to index 11 from dna

In [143]:
dna.groupby('mtdna').agg(
    list
)

,Unnamed: 0,nstr,ystr,gender,approx
mtdna,,,,,
0;1;1;1;1;2;w1;1;1;1,[10],"[(10,11);(29,29);(10,11);(10,10);(15,17);(6,8);(11,12);(12,12);(20,24);(12,14);(14,19);(8,9);(13,13);(8.0,11);(27,28)]",[nan],[F],[Adult]
1;0;0;2;2;2;3;1;1;1,[5],"[(13,13);(30,30);(9,11);(12,13);(15,17);(7,9);(9,12);(11,11);(18,23);(13,13);(16,18);(11,12);(15,17);(11,12);(22,25)]","[16;12;24;29;17.0;15;18,18;13;11;11;23;13;22;16;12;20,21]",[M],[Adult]
1;1;0;2;1;2;1;0;1;1,[4],"[(13,14);(28,30);(11,11);(10,11);(15,18);(6,6);(12,12);(11,11);(17,23);(12,12);(15,16);(8,9);(12,16);(12,12);(16,20)]","[15;13;22;29;15.0;14;17,18;13;10;12;22;11;22;14;11;21]",[M],[Adult]
1;1;1;1;1;2;7;1;1;1,[13],"[(13,14);(28,30);(8,10);(10,12);(16,18);(6,6);(10,15);(11,13);(18,25);(13,14);(16,19);(8,12);(15,17);(7,13);(24,24)]",[nan],[F],[Adult]
1;1;3;1;1;2;0;1;1;1,"[2, 7, 9]","[(13,14);(30,34);(9,12);(11,12);(15,18);(9,10);(9,12);(9,11);(18,21);(13,14);(17,18);(8,11);(15,17);(12,13);(24,25), (13,13);(30,34);(9,12);(10,12);(17,18);(7,10);(12,12);(9,11);(21,23);(13,14);(16,16);(8,11);(17,19);(11,11);(24,25), (13,14);(30,31);(9,12);(10,12);(17,18);(6,7);(9,12);(9,11);(18,21);(13,14);(16,16);(8,11);(15,17);(11,11);(21,22)]","[nan, nan, 16;12;24;29;17.0;15;18,18;13;11;11;23;13;22;16;12;20,21]","[F, F, M]","[Young Adult, Adolescent, Adolescent]"
1;1;4;1;1;2;5;1;1;1,[6],"[(14,14);(29,35);(10,11);(10,11);(16,18);(6,7);(9,11);(12,13);(17,18);(14,14);(16,17);(8,11);(14,16);(12,12.0);(21,22)]","[16;13;24;29;18.0;14;15,18;13;10;12;22;13;23;14;12;20]",[M],[Adult]
1;2;1;1;2;2;w2;1;1;1,"[1, 3, 8, 12]","[(14,15);(28,30);(7,11);(10,11);(16,16);(7,8);(8,11);(12,12);(18,20);(13,14);(14,15);(8,8);(15,16);(9,11);(20,24), (14,14);(30,35);(7,11);(11,11);(16,18);(6,7);(8,9);(12,13);(18,20);(14,14);(15,17);(8,11);(14,15);(9,12);(20,21), (14,14);(29,30);(7,10);(10,11);(16,16);(7,7);(11,11);(12,12);(17,18);(13,14);(15,16);(8,11);(14,16);(9,12);(20,22), (14,14);(28,29);(7,11);(11,11);(16,16);(7,8);(8,11);(12,12);(17,20);(14,14);(14,16);(8,11);(14,16);(11,12);(20,21)]","[nan, 16;13;24;29;18.0;14;15,18;13;10;12;22;13;23;14;12;20, nan, 16;13;24;29;18.0;14;15,18;13;10;12;22;13;23;14;12;20]","[F, M, F, M]","[Adult, Adolescent, Adolescent, Young Adult]"
1;2;3;1;2;2;1;5;1;1,[14],"[(14,16);(29,30);(8,12);(10,11);(16,16);(6,7);(11,11);(14,14);(19,22);(13,14);(16,16);(8,11);(15,15);(10,12);(23,24)]","[15;13;25;31;17;15;16,16;13;10;13;23;11;21;14;11;21.0]",[M],[Adult]
1;3;3;1;1;2;1;1;1;0,[0],"[(11,12);(30,34);(11,13);(11,12);(16,17);(7,10);(8,12);(10,10);(17,24);(13,14);(15,18);(8,10);(17,17);(7,9);(25,26)]","[15;13;23;29;17.0;15;17,17;13;11;12;23;11;25;14;12;19]",[M],[Adult]


Index 2,7,9 are from the family where at least one child is not blood related to the legal parent;

index 1,3,8,12 are liket the family of three where all 3 children are bio childre to their legal parents

In [142]:
dna.groupby('ystr').agg(
    list
)

,Unnamed: 0,nstr,mtdna,gender,approx
ystr,,,,,
"15;13;22;29;15.0;14;17,18;13;10;12;22;11;22;14;11;21",[4],"[(13,14);(28,30);(11,11);(10,11);(15,18);(6,6);(12,12);(11,11);(17,23);(12,12);(15,16);(8,9);(12,16);(12,12);(16,20)]",[1;1;0;2;1;2;1;0;1;1],[M],[Adult]
"15;13;23;29;17.0;15;17,17;13;11;12;23;11;25;14;12;19",[0],"[(11,12);(30,34);(11,13);(11,12);(16,17);(7,10);(8,12);(10,10);(17,24);(13,14);(15,18);(8,10);(17,17);(7,9);(25,26)]",[1;3;3;1;1;2;1;1;1;0],[M],[Adult]
"15;13;25;31;17;15;16,16;13;10;13;23;11;21;14;11;21.0",[14],"[(14,16);(29,30);(8,12);(10,11);(16,16);(6,7);(11,11);(14,14);(19,22);(13,14);(16,16);(8,11);(15,15);(10,12);(23,24)]",[1;2;3;1;2;2;1;5;1;1],[M],[Adult]
"16;12;24;29;17.0;15;18,18;13;11;11;23;13;22;16;12;20,21","[5, 9]","[(13,13);(30,30);(9,11);(12,13);(15,17);(7,9);(9,12);(11,11);(18,23);(13,13);(16,18);(11,12);(15,17);(11,12);(22,25), (13,14);(30,31);(9,12);(10,12);(17,18);(6,7);(9,12);(9,11);(18,21);(13,14);(16,16);(8,11);(15,17);(11,11);(21,22)]","[1;0;0;2;2;2;3;1;1;1, 1;1;3;1;1;2;0;1;1;1]","[M, M]","[Adult, Adolescent]"
"16;13;24;29;18.0;14;15,18;13;10;12;22;13;23;14;12;20","[3, 6, 12]","[(14,14);(30,35);(7,11);(11,11);(16,18);(6,7);(8,9);(12,13);(18,20);(14,14);(15,17);(8,11);(14,15);(9,12);(20,21), (14,14);(29,35);(10,11);(10,11);(16,18);(6,7);(9,11);(12,13);(17,18);(14,14);(16,17);(8,11);(14,16);(12,12.0);(21,22), (14,14);(28,29);(7,11);(11,11);(16,16);(7,8);(8,11);(12,12);(17,20);(14,14);(14,16);(8,11);(14,16);(11,12);(20,21)]","[1;2;1;1;2;2;w2;1;1;1, 1;1;4;1;1;2;5;1;1;1, 1;2;1;1;2;2;w2;1;1;1]","[M, M, M]","[Adolescent, Adult, Young Adult]"


Index 5 and 9 are likely to be father and son of a family;

index 3, 6, 12, are likely to be of another family;

In [152]:
### Filter only adults and run them through LegalChildrenQuery
for row in manif.query('approx == "Adult"').itertuples():
    dtcv.LegalChildrenQuery(E, row.firstname, row.lastname)

TypeError: LegalChildrenQuery() takes 2 positional arguments but 3 were given

In [ ]:
### Check hospital records of the children
lnames = ['Hickmon', 'Fort']

for row in temp_manif.query('lastname in @lnames')[['firstname', 'lastname']].itertuples():
    print(dtcv.HospitalQuery(E, row.firstname, row.lastname))

In [ ]:
pd.merge(dna, E.persondf[['nstr', 'birth', 'firstname', 'lastname']], on='nstr')

In [81]:
temp_manif.groupby(['firstnameD', 'lastnameD']).agg(
    gender=('gender',list),
    firstname=('firstname', list),
    lastname=('lastname',list),
    pids=('pid',list),
    firstnameM=('firstnameM', set),
    lastnameM=('lastnameM', set)
).reset_index()

,firstnameD,lastnameD,gender,firstname,lastname,pids,firstnameM,lastnameM
0,Brian,Alderman,[F],[Ashanti],[McGary],[90658],{Eldora},{Alderman}
1,Britt,Enger,[F],[Mari],[Hickmon],[89006],{Arvilla},{Enger}
2,Dominick,Wesolowski,[F],[Wanetta],[Brey],[89676],{Joaquina},{Wesolowski}
3,Hipolito,Desalvo,[M],[Rick],[Desalvo],[93245],{Eustolia},{Desalvo}
4,Hobert,Hickmon,"[F, M, F]","[Anneliese, Hobert, Luise]","[Hickmon, Hickmon, Hickmon]","[98964, 99480, 96499]",{Mari},{Hickmon}
5,Kirk,McLendon,[F],[Jacki],[Fort],[89148],{Regan},{Turman}
6,Miguel,Fort,"[M, F, M]","[Arlie, Carmella, Nelson]","[Fort, Fort, Fort]","[97696, 98712, 99761]",{Jacki},{Fort}
7,Rodrick,Spindler,[M],[Shalom],[Spindler],[88465],{Gerald},{Yorke}
8,Sang,Fort,[M],[Miguel],[Fort],[88794],{Sonya},{Pompey}
9,Terrell,Hickmon,[M],[Hobert],[Hickmon],[89397],{Susana},{Hickmon}


In [ ]:
temp_manif.groupby('lastname').agg(
    gender=('gender',list),
    firstname=('firstname', list),
    pids=('pid',list),
    firstnameM=('firstnameM', set),
    lastnameM=('lastnameM', set),
    firstnameD=('firstnameD', set),
    lastnameD=('lastnameD', set)
).reset_index()

,lastname,gender,firstname,pids,firstnameM,lastnameM,firstnameD,lastnameD
0,Brey,[F],[Wanetta],[89676],{Joaquina},{Wesolowski},{Dominick},{Wesolowski}
1,Desalvo,[M],[Rick],[93245],{Eustolia},{Desalvo},{Hipolito},{Desalvo}
2,Fort,"[M, F, F, M, M]","[Arlie, Carmella, Jacki, Miguel, Nelson]","[97696, 98712, 89148, 88794, 99761]","{Regan, Jacki, Sonya}","{Fort, Pompey, Turman}","{Miguel, Sang, Kirk}","{McLendon, Fort}"
3,Hickmon,"[F, M, M, F, F]","[Anneliese, Hobert, Hobert, Luise, Mari]","[98964, 89397, 99480, 96499, 89006]","{Mari, Susana, Arvilla}","{Enger, Hickmon}","{Britt, Terrell, Hobert}","{Enger, Hickmon}"
4,McGary,[F],[Ashanti],[90658],{Eldora},{Alderman},{Brian},{Alderman}
5,Nally,[M],[Quincy],[101489],{nan},{nan},{nan},{nan}
6,Spindler,[M],[Shalom],[88465],{Gerald},{Yorke},{Rodrick},{Spindler}
